In [24]:
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
import time
import pandas as pd

In [25]:
def get_all_business_link_in_story(story_url):
    """ This is the function that takes a particular story link from the website (black business business) and then scrolls through all business under
    this story, collects the business links and returns it"""
    # I used chrome; you can use whichever browser you like.
    browser = webdriver.Chrome()
    # Tell Selenium to get the URL you're interested in.
    browser.get(story_url)

    # Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

    # Now that the page is fully scrolled, grab the source code.
    source_data = browser.page_source

    # Throw your source into BeautifulSoup and start parsing!
    bsObj = BeautifulSoup(source_data)

    # Find all business url links in this particular story url
    url_tags = bsObj.find_all('a', {'class': 'u-url'})
    # compile a list of these url
    business_url_links = []
    for tag in url_tags:
        href = tag['href']
        business_url_links.append(href)
    return business_url_links

def getTitle(url_link):
    try:
        html = urlopen(url_link)
        bsObj = BeautifulSoup(html.read(), features="html.parser")
        title = bsObj.title
    except HTTPError as e:
        return None    
    except AttributeError as e:
        return None
    return title.text

def getBusinessInfo(url):
    try:
        html = urlopen(url)
        bsObj = BeautifulSoup(html.read(), features="html.parser")
        business_info = bsObj.find_all('em')
        business_info_text = [info.text for info in business_info]
    except HTTPError:
        return None
    except AttributeError:
        return None
    return business_info_text

def get_all_business_details_in_story(story_url):
    #story_url = "https://www.blackownedbrooklyn.com/stories?category=History%20and%20Culture"
    story_business_url_links = get_all_business_link_in_story(story_url)
    black_business_info = []
    for url in story_business_url_links:
        url = "https://www.blackownedbrooklyn.com" + url
        title = getTitle(url_link=url)
        business_info = getBusinessInfo(url=url)
        black_business = {"business_url": url, "name of business": title, "business contact and address": business_info}
        black_business_info.append(black_business)
        print(title)
        print(business_info)
        print("----------")
    return pd.DataFrame(black_business_info)


In [26]:
history_and_culture = get_all_business_details_in_story("https://www.blackownedbrooklyn.com/stories?category=History%20and%20Culture")

Kokomo — Black-Owned Brooklyn
['“', '65 Kent Ave, Brooklyn, ', '347- 799-1312', ', ', 'kokomonyc.com']
----------
Legacy — Black-Owned Brooklyn
['“We don’t preserve our culture as well as other people preserve it. So, for me, this store is about preserving culture', '247 Water St  Suite 104, Brooklyn, New York, 718- 210-2144, ', 'legacy dumbo.com']
----------
Juneteenth 2021 — Black-Owned Brooklyn
[]
----------
Hattie Carthan — Black-Owned Brooklyn
['more than 1,500', 'Magnolia Tree Earth Center, 677 Lafayette Ave, 718-387-2116, ', 'magnoliatreeearthcenter.com', 'Hattie Carthan Community Garden, 654A Lafayette Ave, 718-638-3566, ', 'hattiecarthancommunitymarket.com']
----------
Origins of AfroPunk  — Black-Owned Brooklyn
[]
----------
Juneteenth 2020 — Black-Owned Brooklyn
[]
----------
Tribute to the Ancestors of the Middle Passage — Black-Owned Brooklyn
[]
----------
None
['bklynboihood.com']
----------
Here's to the Protesters (Black Lives Matter) — Black-Owned Brooklyn
None
-------

In [27]:
history_and_culture

,business_url,name of business,business contact and address
0,https://www.blackownedbrooklyn.com/stories/kokomo,Kokomo — Black-Owned Brooklyn,"[“, 65 Kent Ave, Brooklyn, , 347- 799-1312, , ..."
1,https://www.blackownedbrooklyn.com/stories/legacy,Legacy — Black-Owned Brooklyn,[“We don’t preserve our culture as well as oth...
2,https://www.blackownedbrooklyn.com/stories/jun...,Juneteenth 2021 — Black-Owned Brooklyn,[]
3,https://www.blackownedbrooklyn.com/stories/hat...,Hattie Carthan — Black-Owned Brooklyn,"[more than 1,500, Magnolia Tree Earth Center, ..."
4,https://www.blackownedbrooklyn.com/stories/afr...,Origins of AfroPunk — Black-Owned Brooklyn,[]
5,https://www.blackownedbrooklyn.com/stories/jun...,Juneteenth 2020 — Black-Owned Brooklyn,[]
6,https://www.blackownedbrooklyn.com/stories/tri...,Tribute to the Ancestors of the Middle Passage...,[]
7,https://www.blackownedbrooklyn.com/stories/bkl...,None,[bklynboihood.com]
8,https://www.blackownedbrooklyn.com/stories/her...,Here's to the Protesters (Black Lives Matter) ...,None
9,https://www.blackownedbrooklyn.com/stories/llo...,Remembering Lloyd Porter — Black-Owned Brooklyn,[]


In [28]:
history_and_culture.to_csv(r"C:\Users\udeez\OneDrive\Desktop\MINE\NOIR LABS/history_and_culture.csv", encoding="cp1252", index=False)

In [41]:
food_and_drink = get_all_business_details_in_story("https://www.blackownedbrooklyn.com/stories?category=Food%20and%20Drink")

Dept of Culture — Black-Owned Brooklyn
['“It’s important for us to cook our own food. We need to celebrate our culture from our perspective.” ', '327 Nostrand Ave., Brooklyn, ', 'deptofculturebk.com']
----------
Good Wine — Black-Owned Brooklyn
['327 5th Ave, Brooklyn, 718-499-2392, ', 'goodwine.nyc']
----------
Brooklyn Sweet Spot — Black-Owned Brooklyn
['We are a community business. It’s my bakery, but it’s really *our* bakery.', ' ', 'thebrooklynsweetspot.com']
----------
The Narativ House x Brooklyn Kettle — Black-Owned Brooklyn
['385 Tompkins Ave., 718-676-5154, ', 'thenarativ.com', ', ', 'brooklynkettle.com']
----------
BierWax — Black-Owned Brooklyn
[' 347-533-8449, ', 'bierwaxnyc.com']
----------
Kelewele — Black-Owned Brooklyn
['Dekalb Market Hall, 445 Albee Square W,, 929-322-3480, ', 'kelewelenyc.com']
----------
Mac Shack — Black-Owned Brooklyn
['320 Malcolm X Blvd., 718-210-9603, ', 'macshacknyc.com']
----------
Zanmi — Black-Owned Brooklyn
['1206 Nostrand Ave., 718-676-10

In [42]:
food_and_drink

,business_url,name of business,business contact and address
0,https://www.blackownedbrooklyn.com/stories/dep...,Dept of Culture — Black-Owned Brooklyn,[“It’s important for us to cook our own food. ...
1,https://www.blackownedbrooklyn.com/stories/goo...,Good Wine — Black-Owned Brooklyn,"[327 5th Ave, Brooklyn, 718-499-2392, , goodwi..."
2,https://www.blackownedbrooklyn.com/stories/bro...,Brooklyn Sweet Spot — Black-Owned Brooklyn,"[We are a community business. It’s my bakery, ..."
3,https://www.blackownedbrooklyn.com/stories/the...,The Narativ House x Brooklyn Kettle — Black-Ow...,"[385 Tompkins Ave., 718-676-5154, , thenarativ..."
4,https://www.blackownedbrooklyn.com/stories/bie...,BierWax — Black-Owned Brooklyn,"[ 347-533-8449, , bierwaxnyc.com]"
...,...,...,...
74,https://www.blackownedbrooklyn.com/stories/caf...,Cafe Con Libros — Black-Owned Brooklyn,"[Freshwater, This Will Be My Undoing, This sh*..."
75,https://www.blackownedbrooklyn.com/stories/buka,Buka — Black-Owned Brooklyn,"[ is , 946 Fulton Street, 347-763-0619, , buka..."
76,https://www.blackownedbrooklyn.com/stories/bre...,Breukelen Coffee House — Black-Owned Brooklyn,"[764 Franklin Avenue, 718-789-7070, , breukele..."
77,https://www.blackownedbrooklyn.com/stories/201...,Jemz — Black-Owned Brooklyn,"[everything, even the side salad, 1171 St John..."


In [44]:
food_and_drink.to_csv(r"C:\Users\udeez\OneDrive\Desktop\MINE\NOIR LABS/food_and_drink.csv", encoding="cp1252", index=False)

In [45]:
home_and_design = get_all_business_details_in_story("https://www.blackownedbrooklyn.com/stories?category=Home%20and%20Design")

The Narativ House x Brooklyn Kettle — Black-Owned Brooklyn
['385 Tompkins Ave., 718-676-5154, ', 'thenarativ.com', ', ', 'brooklynkettle.com']
----------
Adanne — Black-Owned Brooklyn
['53 Bridge Street, ', 'adanne.co']
----------
The Regal Suite — Black-Owned Brooklyn
['The Regal Suite ', '(']
----------
YÚ Interiors — Black-Owned Brooklyn
[' interesting', '15 Greene Ave, Brooklyn, 718-237-5878, ']
----------
Photodom — Black-Owned Brooklyn
['1717 Broadway #310, Brooklyn', ' 347-365-5660, ', 'Photodom.shop']
----------
Hardware 2.0 — Black-Owned Brooklyn
['would', 'has', '860 Atlantic Ave, Brooklyn, 347-663-4603, ', 'hardware2bk.com']
----------
Make Manifest — Black-Owned Brooklyn
['if ', '382 Tompkins Ave, ', 'www.makemanifestbk.com']
----------
Brooklyn Clay Industries — Black-Owned Brooklyn
['63 Flushing Ave., Building 62, 301 395-0143, ', 'brooklynclayindustries.com']
----------
Ethel's Club — Black-Owned Brooklyn
['315 Meserole Street, ', 'ethelsclub.com']
----------
The Brownst

In [46]:
home_and_design

,business_url,name of business,business contact and address
0,https://www.blackownedbrooklyn.com/stories/the...,The Narativ House x Brooklyn Kettle — Black-Ow...,"[385 Tompkins Ave., 718-676-5154, , thenarativ..."
1,https://www.blackownedbrooklyn.com/stories/adanne,Adanne — Black-Owned Brooklyn,"[53 Bridge Street, , adanne.co]"
2,https://www.blackownedbrooklyn.com/stories/the...,The Regal Suite — Black-Owned Brooklyn,"[The Regal Suite , (]"
3,https://www.blackownedbrooklyn.com/stories/yui...,YÚ Interiors — Black-Owned Brooklyn,"[ interesting, 15 Greene Ave, Brooklyn, 718-23..."
4,https://www.blackownedbrooklyn.com/stories/pho...,Photodom — Black-Owned Brooklyn,"[1717 Broadway #310, Brooklyn, 347-365-5660, ..."
5,https://www.blackownedbrooklyn.com/stories/har...,Hardware 2.0 — Black-Owned Brooklyn,"[would, has, 860 Atlantic Ave, Brooklyn, 347-6..."
6,https://www.blackownedbrooklyn.com/stories/mak...,Make Manifest — Black-Owned Brooklyn,"[if , 382 Tompkins Ave, , www.makemanifestbk.com]"
7,https://www.blackownedbrooklyn.com/stories/bro...,Brooklyn Clay Industries — Black-Owned Brooklyn,"[63 Flushing Ave., Building 62, 301 395-0143, ..."
8,https://www.blackownedbrooklyn.com/stories/eth...,Ethel's Club — Black-Owned Brooklyn,"[315 Meserole Street, , ethelsclub.com]"
9,https://www.blackownedbrooklyn.com/stories/the...,The Brownstoners of Bedford-Stuyvesant — Black...,None


In [47]:
home_and_design.to_csv(r"C:\Users\udeez\OneDrive\Desktop\MINE\NOIR LABS/home_and_design.csv", encoding="cp1252", index=False)

In [48]:
style_and_beauty = get_all_business_details_in_story("https://www.blackownedbrooklyn.com/stories?category=Style%20and%20Beauty")

The Narativ House x Brooklyn Kettle — Black-Owned Brooklyn
['385 Tompkins Ave., 718-676-5154, ', 'thenarativ.com', ', ', 'brooklynkettle.com']
----------
Paperboy Love Prince  — Black-Owned Brooklyn
['1254 Myrtle Avenue., Brooklyn, ']
----------
Savant Studios — Black-Owned Brooklyn
['all', '917- 868-4923', 'Savantvision.com']
----------
Style Eyes Optical  — Black-Owned Brooklyn
['1005 Flatbush Ave., 718-942-5957, ', 'styleeyesoptical.com']
----------
Browstress — Black-Owned Brooklyn
['68 Jay St, Brooklyn,', ' 917-474-2366, ', 'Browstress.com']
----------
None
[' Brooklyn, 347-295-0001, ', 'auntsetuncles.com']
----------
Da Spot — Black-Owned Brooklyn
['the spot', '445 Albee Square West, 347-903-6729, ', 'daspotnyc.com']
----------
The Wrap Life — Black-Owned Brooklyn
[]
----------
BGLH Marketplace — Black-Owned Brooklyn
[]
----------
Moshood 25th Anniversary Fashion Show — Black-Owned Brooklyn
['1360 Fulton Street, 718-243-9433, ', 'afrikanspirit.com']
----------
None
['Haitian', '4

In [36]:
style_and_beauty

,business_url,name of business,business contact and address
0,https://www.blackownedbrooklyn.com/stories/the...,The Narativ House x Brooklyn Kettle — Black-Ow...,"[385 Tompkins Ave., 718-676-5154, , thenarativ..."
1,https://www.blackownedbrooklyn.com/stories/pap...,Paperboy Love Prince — Black-Owned Brooklyn,"[1254 Myrtle Avenue., Brooklyn, ]"
2,https://www.blackownedbrooklyn.com/stories/sav...,Savant Studios — Black-Owned Brooklyn,None
3,https://www.blackownedbrooklyn.com/stories/sty...,Style Eyes Optical — Black-Owned Brooklyn,"[1005 Flatbush Ave., 718-942-5957, , styleeyes..."
4,https://www.blackownedbrooklyn.com/stories/bro...,None,"[68 Jay St, Brooklyn,, 917-474-2366, , Browst..."
5,https://www.blackownedbrooklyn.com/stories/aun...,Aunts et Uncles — Black-Owned Brooklyn,"[ Brooklyn, 347-295-0001, , auntsetuncles.com]"
6,https://www.blackownedbrooklyn.com/stories/da-...,Da Spot — Black-Owned Brooklyn,"[the spot, 445 Albee Square West, 347-903-6729..."
7,https://www.blackownedbrooklyn.com/stories/the...,The Wrap Life — Black-Owned Brooklyn,[]
8,https://www.blackownedbrooklyn.com/stories/bgl...,BGLH Marketplace — Black-Owned Brooklyn,[]
9,https://www.blackownedbrooklyn.com/stories/mos...,Moshood 25th Anniversary Fashion Show — Black-...,"[1360 Fulton Street, 718-243-9433, , afrikansp..."


In [49]:
style_and_beauty.to_csv(r"C:\Users\udeez\OneDrive\Desktop\MINE\NOIR LABS/style_and_beauty.csv", encoding="cp1252", index=False)

In [50]:
health_and_wellness = get_all_business_details_in_story("https://www.blackownedbrooklyn.com/stories?category=Health%20and%20Wellness")

Lavender Blues — Black-Owned Brooklyn
['175 Malcolm X Blvd', ' Brooklyn, 929-400-1436,', ' ', 'Lavender Blues']
----------
Life Wellness Center — Black-Owned Brooklyn
['376 Tompkins Ave, Brooklyn, 347-461-9939,', ' lifewellnesscenter.life']
----------
Aunts et Uncles — Black-Owned Brooklyn
[' Brooklyn, 347-295-0001, ', 'auntsetuncles.com']
----------
None
['Garden In The Wake']
----------
Brooklyn Supported Agriculture — Black-Owned Brooklyn
['\ufeff']
----------
Ancient Song Doula Services — Black-Owned Brooklyn
['521 Halsey St, Brooklyn, 347-480-9504, ']
----------
Buy Better Foods — Black-Owned Brooklyn
['I’m especially happy that my community is here in this space', '372 Kosciuszko St, Brooklyn, 347-365-9550, ']
----------
We Run Kings  — Black-Owned Brooklyn
['werunkings.com ']
----------
Ras Plant Based — Black-Owned Brooklyn
['739 Franklin Ave, Brooklyn, 718-622-6220, ', 'rasplantbased.com']
----------
Nûrish — Black-Owned Brooklyn
['637 Washington Ave, Brooklyn, 718-484-1122, '

In [51]:
health_and_wellness

,business_url,name of business,business contact and address
0,https://www.blackownedbrooklyn.com/stories/lav...,Lavender Blues — Black-Owned Brooklyn,"[175 Malcolm X Blvd, Brooklyn, 929-400-1436,,..."
1,https://www.blackownedbrooklyn.com/stories/lif...,Life Wellness Center — Black-Owned Brooklyn,"[376 Tompkins Ave, Brooklyn, 347-461-9939,, l..."
2,https://www.blackownedbrooklyn.com/stories/aun...,Aunts et Uncles — Black-Owned Brooklyn,"[ Brooklyn, 347-295-0001, , auntsetuncles.com]"
3,https://www.blackownedbrooklyn.com/stories/gar...,None,[Garden In The Wake]
4,https://www.blackownedbrooklyn.com/stories/bro...,Brooklyn Supported Agriculture — Black-Owned B...,[﻿]
5,https://www.blackownedbrooklyn.com/stories/anc...,Ancient Song Doula Services — Black-Owned Broo...,"[521 Halsey St, Brooklyn, 347-480-9504, ]"
6,https://www.blackownedbrooklyn.com/stories/buy...,Buy Better Foods — Black-Owned Brooklyn,[I’m especially happy that my community is her...
7,https://www.blackownedbrooklyn.com/stories/we-...,We Run Kings — Black-Owned Brooklyn,[werunkings.com ]
8,https://www.blackownedbrooklyn.com/stories/ras...,Ras Plant Based — Black-Owned Brooklyn,"[739 Franklin Ave, Brooklyn, 718-622-6220, , r..."
9,https://www.blackownedbrooklyn.com/stories/nurish,Nûrish — Black-Owned Brooklyn,"[637 Washington Ave, Brooklyn, 718-484-1122, ,..."


In [52]:
health_and_wellness.to_csv(r"C:\Users\udeez\OneDrive\Desktop\MINE\NOIR LABS/health_and_wellness.csv", encoding="cp1252", index=False)